In [1]:
%matplotlib inline
import gluonbook as gb
from mxnet import autograd, nd

3.6.1 获取和读取数据

In [2]:
batch_size = 256
train_iter, test_iter = gb.load_data_fashion_mnist(batch_size)

3.6.2 初始化模型参数

In [4]:
#输入为28*28,类别为10，因此输入维度为784x10,输出为10x1
num_inputs = 784
num_outputs = 10
W = nd.random.normal(scale=0.01, shape=(num_inputs, num_outputs))
b = nd.zeros(num_outputs)
W.attach_grad()
b.attach_grad()

3.6.3 实现Softmax 运算

In [5]:
def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(axis=1, keepdims=True)
    return X_exp / partition 

In [6]:
X = nd.random.normal(shape=(2, 5))
X_prob = softmax(X)
X_prob, X_prob.sum(axis=1)

(
 [[0.11544233 0.10826854 0.27056506 0.40623596 0.09948817]
  [0.05232305 0.21188462 0.14647925 0.20740628 0.38190684]]
 <NDArray 2x5 @cpu(0)>,
 
 [1. 1.]
 <NDArray 2 @cpu(0)>)

3.6.4 定义模型

In [7]:
def net(X):
    return softmax(nd.dot(X.reshape((-1, num_inputs)), W) + b)

3.6.5 定义损失函数

In [8]:
y_hat = nd.array([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = nd.array([0, 2])
nd.pick(y_hat, y)


[0.1 0.5]
<NDArray 2 @cpu(0)>

In [9]:
def cross_entropy(y_hat, y):
    return - nd.pick(y_hat, y).log()

3.6.6 计算分类准确率

In [3]:
def accuracy(y_hat, y):
    return (y_hat.argmax(axis=1) == y.astype('float32')).mean().asscalar() #取最大的位置所在的索引

In [4]:
def evaluate_accuracy(data_iter, net):
    acc = 0
    for X, y in data_iter:
        acc += accuracy(net(X), y)
    return acc / len(data_iter)

3.6.7 训练模型

In [6]:
num_epochs, lr = 5, 0.1

def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
params=None, lr=None, trainer=None):
    for epoch in range(num_epochs):
        train_l_sum = 0
        train_acc_sum = 0
        for X, y in train_iter:
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y)
            l.backward()
            if trainer is None:
                gb.sgd(params, lr, batch_size)
            else:
                trainer.step(batch_size) # 下⼀节将⽤到。
            train_l_sum += l.mean().asscalar()
            train_acc_sum += accuracy(y_hat, y)
        
        test_acc = evaluate_accuracy(test_iter, net)

        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'% (epoch + 1,    train_l_sum / len(train_iter),train_acc_sum / len(train_iter), test_acc))
train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs,batch_size, [W, b], lr)






NameError: name 'net' is not defined